In [1]:
import stlearn as st
from pathlib import Path
import scanpy as sc
st.settings.set_figure_params(dpi=300)
import matplotlib.pyplot as plt 
import pandas as pd
import anndata
from anndata import AnnData
from typing import Optional
import scanpy as sc
import seaborn as sns
import PIL
PIL.Image.MAX_IMAGE_PIXELS = None

In [2]:
import matplotlib.pyplot as plt
from libpysal.weights.contiguity import Queen
from libpysal import examples
import numpy as np
import pandas as pd
import geopandas as gpd
import os
import splot
from splot.esda import moran_scatterplot, lisa_cluster
from esda.moran import Moran, Moran_Local
from esda.moran import Moran_BV, Moran_Local_BV
from splot.esda import plot_moran_bv_simulation, plot_moran_bv, plot_local_autocorrelation

In [3]:
from anndata import AnnData
from typing import Iterable, Union, Optional
import pandas as pd
def spatial_autocorr(adata_true: AnnData,
                     adata_pred: AnnData,
                     model_name: str,
                     p: float = 0.05,
                     save_plots: str = None,
) -> Optional[AnnData]:
    
    assert len(adata_true.var_names) == len(adata_pred.var_names)
    
    gpd_name = "gpd_{}".format(model_name)
    library_id = list(adata_true.uns["spatial"].keys())[0]
    tissue_image = adata_true.uns["spatial"][library_id]["images"]["fulres"]
    
    adata_true.obsm[gpd_name] = gpd.GeoDataFrame(adata_true.obs,
                                                 geometry=gpd.points_from_xy(
                                                          adata_true.obs.imagecol, 
                                                          adata_true.obs.imagerow))
    w = Queen.from_dataframe(adata_true.obsm[gpd_name])
    
    spatial_autocorr_label = []
    moran_i = []
    n_sigs = []
    for gene in adata_true.var_names:
        x = np.array(adata_true.to_df()[gene].values, dtype='float')
        y = np.array(adata_pred.to_df()[gene].values, dtype='float')
        
        adata_true.obsm[gpd_name]["gc_{}".format(gene)] = x
        adata_true.obsm[gpd_name]["pred_{}".format(gene)] = y
        
#         moran = Moran(x,w)
        moran_bv = Moran_BV(y, x, w)
#         moran_loc = Moran_Local(x, w)
        moran_loc_bv = Moran_Local_BV(y, x, w)
        
        labels, n_sig = hot_cold_label(moran_loc_bv, p)
        spatial_autocorr_label.append(labels)
        moran_i.append(moran_bv.I)
        n_sigs.append(n_sig)
        
        if save_plots:
            lisa_cluster(moran_loc_bv, adata_true.obsm[gpd_name], p=0.05, 
                         figsize = (9,9), markersize=12,)# **{"alpha":1})
            plt.imshow(tissue_image)
            plt.savefig(save_plots / "lisa_cluster_{}".format(gene))
            
        adata_true.obsm[gpd_name].drop(["gc_{}".format(gene),
                                        "pred_{}".format(gene)], inplace=True, axis=1)
    
    label_array = np.array(spatial_autocorr_label).transpose()
    
    n_zero = (adata_true.to_df() == 0).apply(sum, axis=0) / len(adata_true.obs_names)
    df = pd.DataFrame({
        "moran_i":moran_i,
        "n_sigs":n_sigs,
        "n_zero":n_zero
    }, index=adata_true.var_names)
    return df


def hot_cold_label(moran_loc, p):
    cluster = moran_hot_cold_spots(moran_loc, p)
    cluster_labels = ['ns', 'HH', 'LH', 'LL', 'HL']
    labels = [cluster_labels[i] for i in cluster]
    n_sig = labels.count('HH') + labels.count('LL')
    return labels, n_sig

In [4]:
def pearson_r2(adata_true: AnnData,
                     adata_pred: AnnData,
                     model_name: str,
                     p: float = 0.05,
                     save_plots: str = None,
) -> Optional[AnnData]:
    
    assert len(adata_true.var_names) == len(adata_pred.var_names)
    
    gpd_name = "gpd_{}".format(model_name)
    library_id = list(adata_true.uns["spatial"].keys())[0]
    tissue_image = adata_true.uns["spatial"][library_id]["images"]["fulres"]
    
    pearson_r_list =[]
    for gene in adata_true.var_names:
        x = np.array(adata_true.to_df()[gene].values, dtype='float')
        y = np.array(adata_pred.to_df()[gene].values, dtype='float')
        
        r_value = stats.pearsonr(x, 
                                 y)[0] **2
        pearson_r_list.append(r_value)
    
    label_array = np.array(pearson_r_list).transpose()
    
    n_zero = (adata_true.to_df() == 0).apply(sum, axis=0) / len(adata_true.obs_names)
    df = pd.DataFrame({
        "r2":pearson_r_list,
        "n_zero":n_zero
    }, index=adata_true.var_names)
    return df

In [5]:
from matplotlib import patches, colors
def lisa_cluster(moran_loc, gdf, p=0.05, ax=None,
                 legend=True, legend_kwds=None, **kwargs):
    """
    Create a LISA Cluster map
    Parameters
    ----------
    moran_loc : esda.moran.Moran_Local or Moran_Local_BV instance
        Values of Moran's Local Autocorrelation Statistic
    gdf : geopandas dataframe instance
        The Dataframe containing information to plot. Note that `gdf` will be
        modified, so calling functions should use a copy of the user
        provided `gdf`. (either using gdf.assign() or gdf.copy())
    p : float, optional
        The p-value threshold for significance. Points will
        be colored by significance.
    ax : matplotlib Axes instance, optional
        Axes in which to plot the figure in multiple Axes layout.
        Default = None
    legend : boolean, optional
        If True, legend for maps will be depicted. Default = True
    legend_kwds : dict, optional
        Dictionary to control legend formatting options. Example:
        ``legend_kwds={'loc': 'upper left', 'bbox_to_anchor': (0.92, 1.05)}``
        Default = None
    **kwargs : keyword arguments, optional
        Keywords designing and passed to geopandas.GeoDataFrame.plot().
    Returns
    -------
    fig : matplotlip Figure instance
        Figure of LISA cluster map
    ax : matplotlib Axes instance
        Axes in which the figure is plotted
    Examples
    --------
    Imports
    
    >>> import matplotlib.pyplot as plt
    >>> from libpysal.weights.contiguity import Queen
    >>> from libpysal import examples
    >>> import geopandas as gpd
    >>> from esda.moran import Moran_Local
    >>> from splot.esda import lisa_cluster
    Data preparation and statistical analysis
    
    >>> guerry = examples.load_example('Guerry')
    >>> link_to_data = guerry.get_path('guerry.shp')
    >>> gdf = gpd.read_file(link_to_data)
    >>> y = gdf['Donatns'].values
    >>> w = Queen.from_dataframe(gdf)
    >>> w.transform = 'r'
    >>> moran_loc = Moran_Local(y, w)
    Plotting
    
    >>> fig = lisa_cluster(moran_loc, gdf)
    >>> plt.show()
    
    """
    # retrieve colors5 and labels from mask_local_auto
    _, colors5, _, labels = mask_local_auto(moran_loc, p=p)

    # define ListedColormap
    hmap = colors.ListedColormap(colors5)

    if ax is None:
        figsize = kwargs.pop('figsize', None)
        fig, ax = plt.subplots(1, figsize=figsize)
    else:
        fig = ax.get_figure()

    gdf.assign(cl=labels).plot(column='cl', categorical=True,
                               k=2, cmap=hmap, linewidth=0.1, ax=ax,
                               edgecolor='white', legend=legend,
                               legend_kwds=legend_kwds, **kwargs)
    ax.set_axis_off()
    ax.set_aspect('equal')
    return fig, ax


def mask_local_auto(moran_loc, p=0.5):
    '''
    Create Mask for coloration and labeling of local spatial autocorrelation
    Parameters
    ----------
    moran_loc : esda.moran.Moran_Local instance
        values of Moran's I Global Autocorrelation Statistic
    p : float
        The p-value threshold for significance. Points will
        be colored by significance.
    Returns
    -------
    cluster_labels : list of str
        List of labels - ['ns', 'HH', 'LH', 'LL', 'HL']
    colors5 : list of str
        List of colours - ['#d7191c', '#fdae61', '#abd9e9',
        '#2c7bb6', 'lightgrey']
    colors : array of str
        Array containing coloration for each input value/ shape.
    labels : list of str
        List of label for each attribute value/ polygon.
    '''
    # create a mask for local spatial autocorrelation
    cluster = moran_hot_cold_spots(moran_loc, p)

    cluster_labels = ['ns', 'HH', 'LH', 'LL', 'HL']
    labels = [cluster_labels[i] for i in cluster]

    colors5 = {0: '#ffffff00',
               1: '#d7191cff',
               2: '#abd9e9ff',
               3: '#2c7bb6ff',
               4: '#fdae61ff'}
    colors = [colors5[i] for i in cluster]  # for Bokeh
    # for MPL, keeps colors even if clusters are missing:
    x = np.array(labels)
    y = np.unique(x)
    colors5_mpl = {'HH': '#d7191cff',
                   'LH': '#abd9e9ff',
                   'LL': '#2c7bb6ff',
                   'HL': '#fdae61ff',
                   'ns': '#ffffff00'}
    colors5 = [colors5_mpl[i] for i in y]  # for mpl

    # HACK need this, because MPL sorts these labels while Bokeh does not
    cluster_labels.sort()
    return cluster_labels, colors5, colors, labels


def moran_hot_cold_spots(moran_loc, p=0.05):
    sig = 1 * (moran_loc.p_sim < p)
    HH = 1 * (sig * moran_loc.q == 1)
    LL = 3 * (sig * moran_loc.q == 3)
    LH = 2 * (sig * moran_loc.q == 2)
    HL = 4 * (sig * moran_loc.q == 4)
    cluster = HH + LL + LH + HL
    return cluster

In [6]:
from scipy import stats

def plot_correlation(df, attr_1, attr_2):
    r = stats.pearsonr(df[attr_1], 
                       df[attr_2])[0] **2

    g = sns.lmplot(data=df,
        x=attr_1, y=attr_2,
        height=5, legend=True
    )
    # g.set(ylim=(0, 360), xlim=(0,360))

    g.set_axis_labels(attr_1, attr_2)
    plt.annotate(r'$R^2:{0:.2f}$'.format(r),
                (max(df[attr_1])*0.9, max(df[attr_2])*0.9))
    return g

In [7]:
def merge(
        adata1: AnnData,
        adata2: AnnData,
        copy: bool = True,

) -> Optional[AnnData]:
    merged_df = adata1.to_df().append(adata2.to_df())
    merged_df_obs = adata1.obs.append(adata2.obs)
    merged_adata = AnnData(merged_df, obs=merged_df_obs)
    merged_adata.uns["spatial"] = adata1.uns["spatial"]
    
    return merged_adata if copy else None

In [8]:
BASE_PATH = Path("/home/uqxtan9/90days")
TILE_PATH = Path("/tmp/tiles")
TILE_PATH.mkdir(parents=True, exist_ok=True)

In [9]:
OUT_PATH = Path("/home/uqxtan9/90days/stlearn_revision2/imputation_bc")
OUT_PATH.mkdir(parents=True, exist_ok=True)

In [10]:
SAMPLE = "block1"
Sample1 = st.Read10X(BASE_PATH / "breast_cancer_10x", 
                     library_id=SAMPLE,
                     quality="fulres",)
img = plt.imread(BASE_PATH / "breast_cancer_10x" /"V1_Breast_Cancer_Block_A_Section_1_image.tif", 0)
Sample1.uns["spatial"][SAMPLE]['images']["fulres"] = img

Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.


In [11]:
SAMPLE = "block2"
Sample2 = st.Read10X(BASE_PATH / "breast_cancer_2_10x", 
                     library_id=SAMPLE,
                     quality="fulres",)
img = plt.imread(BASE_PATH / "breast_cancer_2_10x" /"V1_Breast_Cancer_Block_A_Section_2_image.tif", 0)
Sample2.uns["spatial"][SAMPLE]['images']["fulres"] = img

Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.


In [12]:
for andata in [
    Sample1, 
    Sample2
]:
    st.pp.filter_genes(andata,min_cells=3)
    st.pp.normalize_total(andata)
    st.pp.log1p(andata)
#     st.pp.scale(andata)
    st.em.run_pca(andata,n_comps=50)
    TILE_PATH_ = TILE_PATH / list(Sample1.uns["spatial"].keys())[0]
    st.pp.tiling(andata, TILE_PATH_, crop_size=299)
    st.pp.extract_feature(andata)
    st.spatial.SME.SME_normalize(andata, use_data="raw")

Normalization step is finished in adata.X
Log transformation step is finished in adata.X


KeyboardInterrupt: 

In [ ]:
Sample1_imputation = Sample1.copy()
Sample1_imputation.X = Sample1_imputation.obsm["imputed_data"]
# Sample1_imputation.write_h5ad(OUT_PATH/"Sample1_imputation.h5ad")

In [ ]:
Sample2_imputation = Sample2.copy()
Sample2_imputation.X = Sample2_imputation.obsm["imputed_data"]
# Sample2_imputation.write_h5ad(OUT_PATH/"Sample2_imputation.h5ad")

In [12]:
Sample1_imputation = anndata.read_h5ad(OUT_PATH/"Sample1_imputation.h5ad")
Sample2_imputation = anndata.read_h5ad(OUT_PATH/"Sample2_imputation.h5ad")

In [15]:
gene = "COX6C"
model_name = "stSME"
adata_true = Sample1.copy()
adata_pred = Sample1_imputation.copy()
p = 0.05

gpd_name = "gpd_{}".format(model_name)
library_id = list(adata_true.uns["spatial"].keys())[0]
tissue_image = adata_true.uns["spatial"][library_id]["images"]["fulres"]
    
adata_true.obsm[gpd_name] = gpd.GeoDataFrame(adata_true.obs,
                                                 geometry=gpd.points_from_xy(
                                                          adata_true.obs.imagecol, 
                                                          adata_true.obs.imagerow))
w = Queen.from_dataframe(adata_true.obsm[gpd_name])
x = np.array(adata_true.to_df()[gene].values, dtype='float')
y = np.array(adata_pred.to_df()[gene].values, dtype='float')

adata_true.obsm[gpd_name]["gc_{}".format(gene)] = x
adata_true.obsm[gpd_name]["pred_{}".format(gene)] = y
        
#         moran = Moran(x,w)
moran_bv = Moran_BV(y, x, w)
#         moran_loc = Moran_Local(x, w)
moran_loc_bv = Moran_Local_BV(y, x, w)
        
labels, n_sig = hot_cold_label(moran_loc_bv, p)

In [1]:
# lisa_cluster(moran_loc_bv, adata_true.obsm[gpd_name], figsize = (9,9), markersize=12, **{"alpha":0.8})
# plt.imshow(adata_true.uns["spatial"]["block1"]["images"]["fulres"])
# plt.show()

In [2]:
common_genes = Sample1.var_names.intersection(Sample2.var_names)
Sample1_commom = Sample1[:,common_genes].copy()
Sample2_commom = Sample2[:,common_genes].copy()
Sample1_imputation_commom = Sample1_imputation[:,common_genes].copy()
Sample2_imputation_commom = Sample2_imputation[:,common_genes].copy()

In [ ]:
sc.pp.highly_variable_genes(Sample1_commom, n_top_genes=2000)

In [ ]:
df_1_ = spatial_autocorr(Sample1_commom[:,Sample1_commom.var.highly_variable], 
                      Sample1_imputation_commom[:,Sample1_commom.var.highly_variable], "stSME")

In [ ]:
df_2_ = spatial_autocorr(Sample2_commom[:,Sample1_commom.var.highly_variable], 
                      Sample2_imputation_commom[:,Sample1_commom.var.highly_variable], "stSME")

In [ ]:
# df_1_.to_csv(OUT_PATH / "2000HVG_autocorr_S1.csv")
# df_2_.to_csv(OUT_PATH / "2000HVG_autocorr_S2.csv")

In [ ]:
df_1_ = pd.read_csv(OUT_PATH / "2000HVG_autocorr_S1.csv", index_col=0)
df_2_ = pd.read_csv(OUT_PATH / "2000HVG_autocorr_S2.csv", index_col=0)

In [ ]:
df_1_["Sample"] = "Sample1"
df_2_["Sample"] = "Sample2"
df_1_["Genes"] = df_1_.index
df_2_["Genes"] = df_2_.index
df_final = pd.concat([df_1_, df_2_], axis=0,ignore_index=True)

In [ ]:
bins = np.arange(0.0, 1.1, 0.1)
labels = ["<"+str(i)+"%" for i in np.arange(10, 110, 10)]
df_final["bin"] = pd.cut(df_final["n_zero"], bins=bins, labels=labels)

In [ ]:
f, ax = plt.subplots(figsize=(6, 5))
g = sns.scatterplot(x=df_1_["moran_i"], y=df_2_["moran_i"], s=10, hue = df_1_["n_zero"],palette="Reds_r",)
g.legend(loc='center left', bbox_to_anchor=(1, 0.85), ncol=1,
         title='Percentage of "0" Spots')
plt.title('Spatial Autocorrelation')
# Set x-axis label
plt.xlabel('Moran\'s I of Sample 1')
# Set y-axis label
plt.ylabel('Moran\'s I of Sample 2')
plt.plot([0, 0], [1, 1], linewidth=2)
plt.tight_layout()
# plt.savefig(OUT_PATH / "plot_2000HVG_spaAuto.pdf", dpi=300)
plt.show()

In [ ]:
f, ax = plt.subplots(figsize=(6, 5))
g = sns.stripplot(x="Sample", y="moran_i", s=2, hue="bin", palette="Reds_r",
                  data=df_final)
# sns.boxplot(x="Sample", y="moran_i",
#                   data=df_final)
g.legend(loc='center left', bbox_to_anchor=(1, 0.7), ncol=1,
         title='Percentage of "0" Spots')
plt.title('Spatial Autocorrelation')
# Set x-axis label
plt.ylabel('Moran\'s I')
# Set y-axis label
plt.xlabel('Sample')
# plt.plot([0, 0], [1, 1], linewidth=2)
plt.tight_layout()
# plt.savefig(OUT_PATH / "plot_2000HVG_spaAuto.pdf", dpi=300)
plt.show()

In [ ]:
df_1_ = pearson_r2(Sample1_commom[:,Sample1_commom.var.highly_variable], 
                      Sample1_imputation_commom[:,Sample1_commom.var.highly_variable], "stSME")

In [ ]:
df_2_ = pearson_r2(Sample2_commom[:,Sample1_commom.var.highly_variable], 
                      Sample2_imputation_commom[:,Sample1_commom.var.highly_variable], "stSME")

In [ ]:
df_1_.to_csv(OUT_PATH / "2000HVG_pearson_r2_S1.csv")
df_2_.to_csv(OUT_PATH / "2000HVG_pearson_r2_S2.csv")

In [ ]:
df_1_ = pd.read_csv(OUT_PATH / "2000HVG_pearson_r2_S1.csv", index_col=0)
df_2_ = pd.read_csv(OUT_PATH / "2000HVG_pearson_r2_S2.csv", index_col=0)

In [ ]:
df_1_["Sample"] = "Sample1"
df_2_["Sample"] = "Sample2"
df_1_["Genes"] = df_1_.index
df_2_["Genes"] = df_2_.index
df_final = pd.concat([df_1_, df_2_], axis=0,ignore_index=True)

In [ ]:
bins = np.arange(0.0, 1.1, 0.1)
labels = ["<"+str(i)+"%" for i in np.arange(10, 110, 10)]
df_final["bin"] = pd.cut(df_final["n_zero"], bins=bins, labels=labels)

In [ ]:
f, ax = plt.subplots(figsize=(6, 5))
g = sns.scatterplot(x=df_1_["r2"], y=df_2_["r2"], s=10, hue = df_1_["n_zero"],palette="Reds_r",)
g.legend(loc='center left', bbox_to_anchor=(1, 0.85), ncol=1,
         title='Percentage of "0" Spots')
plt.title('pearson correlation')
# Set x-axis label
plt.xlabel('pearson correlation of Sample 1')
# Set y-axis label
plt.ylabel('pearson correlation of Sample 2')
plt.plot([0, 0], [1, 1], linewidth=2)
plt.tight_layout()
# plt.savefig(OUT_PATH / "plot_2000HVG_spaAuto.pdf", dpi=300)
plt.show()